In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
plt.ion()   # interactive mode

from skimage import io

###########################################################################
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224), ##ngau nhien resize sau do crop và resize ảnh thành 224x224
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 
    ]),
    'val': transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'hymenoptera_data'
# Load anh tu folder va thuc hien transform. Voi hai folder la train va val. 
# return(sample, target) where target is class_index of the target class.
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
# MAng chua tap anh train va tap anh validate da duoc load voi size 4 
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
# MAng chua thong tin ve so luong anh trong tap train va so luong anh trong tap val
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
### NOTE what is the purpose of this row?
class_names = image_datasets['train'].classes 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

###########################################################################

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])
###########################################################################

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    #- params:
    ##- criterion: ham do loi
    ##- scheduler:methods to adjust the learning rate based on the number of epochs
    ##- optimizer: phuogn phap toi uu
    ##- epochs
    since = time.time()

    ##-- Copy lại mô hình đã được train trước
    best_model_wts = copy.deepcopy(model.state_dict())#a dictionary containing a whole state of the module
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            ##-- mỗi lần load 4 ảnh và tiến hành học 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                ##-- train process
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            ##-- Nếu độ chính xác tăng thì mô hình được coi là cải thiện -> copy lại
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

###########################################################################
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)



<Figure size 640x480 with 1 Axes>

In [ ]:
###########################################################################
##############################VGG#############################################
### LOAD mô hình đã được train trước VGG 
## reset final fully connected layer.
## adjust the last fully connected layer to the ‘ant’ and ‘bee’ poblem. 
'''
resnet18 = models.resnet18()
alexnet = models.alexnet()
vgg16 = models.vgg16()
squeezenet = models.squeezenet1_0()
densenet = models.densenet161()
inception = models.inception_v3()
or 
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
squeezenet = models.squeezenet1_0(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet161(pretrained=True)
inception = models.inception_v3(pretrained=True)

SEE more https://pytorch.org/docs/stable/torchvision/models.html?highlight=models%20resnet18

----------------------
For VGG16 you would have to use model_ft.classifier. You can find the corresponding code here https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py#L29
Here is a small example how to reset the last layer. Of course you could also replace the whole classifier, if that’s what you wish.

model = models.vgg16(pretrained=False)
model.classifier[-1] = nn.Linear(in_features=4096, out_features=num_classes)

-----
https://www.kaggle.com/carloalbertobarbano/vgg16-transfer-learning-pytorch

'''
## using print(model_ft) for checking the model parameters

model_ft = models.vgg16(pretrained=True)
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(in_features=num_ftrs, out_features=2)
model_ft = model_ft.to(device) ## set device 

criterion = nn.CrossEntropyLoss() 

##The learning rate will start at 0.001 and a StepLR object will decrese it by a factor of 0.1 every 7 epochs (not really needed here as we're training for 2 epochs only)
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
"""Sets the learning rate of each parameter group to the initial lr
    decayed by gamma every step_size epochs. When last_epoch=-1, sets
    initial lr as lr.

    Args:
        optimizer (Optimizer): Wrapped optimizer.
        step_size (int): Period of learning rate decay.
        gamma (float): Multiplicative factor of learning rate decay.
            Default: 0.1.
        last_epoch (int): The index of last epoch. Default: -1.

    Example:
        >>> # Assuming optimizer uses lr = 0.05 for all groups
        >>> # lr = 0.05     if epoch < 30
        >>> # lr = 0.005    if 30 <= epoch < 60
        >>> # lr = 0.0005   if 60 <= epoch < 90
        >>> # ...
        >>> scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
        >>> for epoch in range(100):
        >>>     scheduler.step()
        >>>     train(...)
        >>>     validate(...)
    """
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

###########################################################################
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)
visualize_model(model_ft)



Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\ThinhPCP1/.torch\models\vgg16-397923af.pth
100.0%


Epoch 0/24
----------
train Loss: 0.5002 Acc: 0.7828
val Loss: 0.6638 Acc: 0.7320

Epoch 1/24
----------
train Loss: 0.3606 Acc: 0.8525
val Loss: 0.4313 Acc: 0.8301

Epoch 2/24
----------
train Loss: 0.3473 Acc: 0.8730
val Loss: 0.3852 Acc: 0.8366

Epoch 3/24
----------
train Loss: 0.3870 Acc: 0.8156
val Loss: 0.2976 Acc: 0.8758

Epoch 4/24
----------
train Loss: 0.2285 Acc: 0.8934
val Loss: 0.2509 Acc: 0.9150

Epoch 5/24
----------
train Loss: 0.1451 Acc: 0.9590
val Loss: 0.2353 Acc: 0.9216

Epoch 6/24
----------
train Loss: 0.1368 Acc: 0.9344
val Loss: 0.5705 Acc: 0.8497

Epoch 7/24
----------
train Loss: 0.1846 Acc: 0.9303
val Loss: 0.2678 Acc: 0.9085

Epoch 8/24
----------
train Loss: 0.0828 Acc: 0.9754
val Loss: 0.2842 Acc: 0.9216

Epoch 9/24
----------
train Loss: 0.0770 Acc: 0.9754
val Loss: 0.2928 Acc: 0.9150

Epoch 10/24
----------
train Loss: 0.0594 Acc: 0.9754
val Loss: 0.2822 Acc: 0.9216

Epoch 11/24
----------
train Loss: 0.0714 Acc: 0.9795
val Loss: 0.2753 Acc: 0.9281

Ep

In [ ]:
###########################################################################
##############################ALEXNET ConvNet as fixed feature extractor#############################################

model_conv = torchvision.models.vgg16(pretrained=True)
## model_conv.parameters(): Returns an iterator over module parameters.
for param in model_conv.parameters(): 
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
#num_ftrs = model_conv.fc.in_features
num_ftrs = model_conv.classifier[6].in_features
#model_ft.fc = nn.Linear(num_ftrs, 2) ## set a new model with in features and 2 out features
model_conv.classifier[6] = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
#optimizer_conv = optim.SGD(model_conv.classifier.parameters(), lr=0.001, momentum=0.9)
### From this page to solve the problem: https://github.com/amdegroot/ssd.pytorch/issues/109
optimizer_conv = optim.SGD(filter(lambda p: p.requires_grad, model_conv.classifier.parameters()), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

### TRain and evaluate
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=25)

## Evaluate
visualize_model(model_conv)

plt.ioff()
plt.show()

In [ ]:
###########################
## TEST FOR SINGLE IMAGE
data_dir = 'fortesting_data'
image_name = 'bee1.jpeg'

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])   
    ])
# Load anh tu folder va thuc hien transform. Voi hai folder la train va val. 
# return(sample, target) where target is class_index of the target class.

real_image = io.imread(os.path.join(data_dir, image_name))
plt.imshow(real_image)

#real_image = Image.open((os.path.join(data_dir, image_name)))

pilTrans = transforms.ToPILImage()
input_A = data_transforms(pilTrans(real_image)).unsqueeze_(0)


def img_visuallize(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    #inp = np.clip(inp, 0, 1)
    plt.imshow(inp)

def visualize_singeImage(model, input_A):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    
    outputs = model(input_A)
    _, preds = torch.max(outputs, 1)
    print(preds[0])
    print(class_names[preds[0]])
    print(input_A)
    img_visuallize(input_A.cpu().data[0])

    
##############
visualize_singeImage(model_conv, input_A)